In [1]:
import numpy as np
import gzip, pickle, random

In [2]:
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, LSTM, GRU, Dense, Activation, TimeDistributed
from keras.optimizers import SGD
from keras.utils.np_utils import to_categorical

Using Theano backend.


In [3]:
def load_atis(path, folder=0):
    
    folders = {0: 'atis.fold0.pkl.gz',
               1: 'atis.fold1.pkl.gz',
               2: 'atis.fold2.pkl.gz',
               3: 'atis.fold3.pkl.gz',
               4: 'atis.fold4.pkl.gz'}
    
    f = gzip.open(path+folders[folder], 'rb')
    train, valid, test, dicts = pickle.load(f)
    f.close()
    
    train = (train[0],train[2])
    valid = (valid[0],valid[2])
    test = (test[0],test[2])

    return {'train':train, 'valid':valid, 'test':test, 'dicts':dicts}

# ATIS PATH: /Users/jacobsw/Desktop/IMPLEMENTATION_CAMP/CODE/OJO/LUIS/DATA

In [4]:
#     X_train, Y_train = train[0], train[2]
#     X_valid, Y_valid = valid[0], valid[2]
#     X_test, Y_test = test[0], test[2]
#     vocsize = len(dicts['words2idx'])
#     nlabels = len(dicts['labels2idx'])
#     nsentences = len(X_train)

# data_ft = [ (map(data_transform_fn('x'),x),map(data_transform_fn('y'),y)) 
#             for x,y in [(X_train,Y_train), (X_valid,Y_valid), (X_test,Y_test)] ]
# X_train_ft, Y_train_ft = data_ft[0]
# X_valid_ft, Y_valid_ft = data_ft[1]
# X_test_ft, Y_test_ft = data_ft[2]

In [41]:
class NER:
    
    def __init__(self, data, model_dir, mode='SimpleRNN'):
        '''
        # data: a dictionary which contains ...
            {'train':train, 'valid':valid, 'test':test, 'dicts':dicts}
            each value in the dictionary is a 2-tuple ...
            (encoded_sentences, encoded_labels)
            dicts includes ...
            'words2idx', 'labels2idx'
        # mode: 'SimpleRNN', 'LSTM', 'GRU'.
        '''
        if mode not in ['SimpleRNN','LSTM','GRU']:
            print "MODE ERROR: only 'SimpleRNN', 'LSTM', 'GRU'. \n"
            return
        self.mode = mode
        try:
            self.model_dir = model_dir
        except IOError:
            print "IOError: Check if directory is correct."
        try:
            data_transformed = [ (map(self.__transform(len(data['dicts']['labels2idx']),'x'),x),
                                      map(self.__transform(len(data['dicts']['labels2idx']),'y'),y)) 
                                      for x,y in [data['train'], 
                                                  data['valid'], 
                                                  data['test']] ]
            self.X_train, self.Y_train = data_transformed[0]
            self.X_valid, self.Y_valid = data_transformed[1]
            self.X_test, self.Y_test = data_transformed[2]
            self.dicts = data['dicts']
        except:
            print "DATA FORMAT ERROR: \n", \
                  "data = {'train':train, 'valid':valid, 'test':test, 'dicts':dicts} \n", \
                  "value = (encoded_sentences, encoded_labels) \n"
    
    def __transform(self, nlabels, mode='x'):
        '''
        # x: (sent_len, ) => (1, sent_len)
        # y: (sent_len, ) => (1, sent_len, nlabels)
        '''
        if mode not in ['x','y']: 
            print "MODE ERROR: only 'x' and 'y'. \n"
            return
        if mode=='x': return lambda x: np.asarray([x])  
        if mode=='y': return lambda y: to_categorical(np.asarray(y)[:,np.newaxis],
                                                  nlabels)[np.newaxis,:,:] 
    
    def __shuffle(self, X, Y, seed):
        '''
        # X, Y: data and corresponding labels.
        # seed: ensure the same after-shuffle order for X and Y.
        '''
        random.seed(seed)
        random.shuffle(X)
        random.seed(seed)
        random.shuffle(Y)
    
    def __get_mean_evaluation(self, X, Y):
        losses, accuracies = [], []
        for i in xrange(100):
            loss,accuracy = ner.model.evaluate(X[i],Y[i],verbose=0)
            losses.append(loss)
            accuracies.append(accuracy)
        return (np.mean(losses),np.mean(accuracies))
        
    def train(self, validation=False, validation_freq=1, verbose=0, verbose_freq=100,
                    lr=.1, nhidden=100, emb_dim=100, nepochs=1):
        '''
        # validation, validation_freq: 
            if true, run validation at validation_freq epoch (1 by default).
        # verbose, verbose_freq: 
            0: only print out simple messages (e.g. '... building models').
            1: print out validation too.
            print training progress after training every verbose_freq sentences.
        # lr: learning rate.
        # nhidden: number of hidden neurons.
        # emb_dim: dimension of word embeddings.
        # nepochs: number of epochs.
        '''
        print "... configuring model"
        vocsize = len(self.dicts['words2idx'])
        nlabels = len(self.dicts['labels2idx'])
        nsents = len(self.X_train)
        self.dicts['idx2labels'] = {i:l for l,i in self.dicts['labels2idx'].iteritems()}
        self.dicts['idx2words'] = {i:w for w,i in self.dicts['words2idx'].iteritems()}
            
        print "... building model"            
        self.model = Sequential()
        self.model.add(Embedding(input_dim=vocsize, output_dim=emb_dim))
        if self.mode=='LSTM':
            self.model.add(LSTM(output_dim=nhidden, activation='sigmoid', return_sequences=True))
        elif self.mode=='GRU':
            self.model.add(GRU(output_dim=nhidden, activation='sigmoid', return_sequences=True))
        else: 
            self.model.add(SimpleRNN(output_dim=nhidden, activation='sigmoid', return_sequences=True))
        self.model.add(TimeDistributed(Dense(output_dim=nlabels)))
        self.model.add(Activation('softmax'))
        sgd = SGD(lr=lr, momentum=.0, decay=.0, nesterov=False)
        self.model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
        
        print "... training model"
        patience = 3000
        patience_increase_ratio = 2
        improvement_threshold = .995
        best_validation_loss = np.inf
        best_iter = 0
        best_loss = np.inf
        best_accuracy = 0.
        for e in xrange(nepochs):
            e += 1
            if verbose: print "... Epoch: %d" % e
            self.__shuffle(self.X_train,self.Y_train,seed=0)
            for i in xrange(nsents):
                if verbose and i!=0 and i%verbose_freq==0:
                    print "    ... trained %d sentences" % i
                if self.X_train[i].shape[1]==1: continue
                self.model.train_on_batch(self.X_train[i],self.Y_train[i])
        
                if validation and i%validation_freq==0:
                    sample_idxs = random.sample(range(len(self.X_valid)),100)
                    X_valid_samples = [self.X_valid[k] for k in sample_idxs]
                    Y_valid_samples = [self.Y_valid[k] for k in sample_idxs]
                    avg_loss, avg_accuracy = self.__get_mean_evaluation(X_valid_samples, 
                                                                        Y_valid_samples)
                    current_iter = ((e-1)*nsents+i)
                    if avg_loss < best_validation_loss*improvement_threshold:
                        patience = max(patience, current_iter*patience_increase_ratio)
                        best_validation_loss = avg_loss
                        best_iter = current_iter
                        sample_idxs = random.sample(range(len(self.X_test)),100)
                        X_test_samples = [self.X_test[k] for k in sample_idxs]
                        Y_test_samples = [self.Y_test[k] for k in sample_idxs]
                        avg_test_loss, avg_test_accuracy = self.__get_mean_evaluation(X_test_samples, 
                                                                                      Y_test_samples)
                        print "Validation: Loss = %.6f | Accuracy = %.6f" % (avg_loss, avg_accuracy)
                        print "Test: Loss = %.6f | Accuracy = %.6f" % (avg_test_loss, avg_test_accuracy)
                        if avg_test_loss<best_loss and avg_test_accuracy<best_accuracy:
                            best_loss = avg_test_loss
                            best_accuracy = avg_test_accuracy
                            # TODO: save best model
                    if patience <= current_iter:
                        with open('best_model.pkl', 'wb') as f:
                            pickle.dump(self.model, f)
                        print "TRAINING COMPLETE (at iteration %d)" % current_iter
                        return
                
    def predict(self, x): 
        '''
        # x: sentences in string (e.g. "i'd like to book a flight from london to paris")
        # return: a list of predicted labels associated with the words in the sentence x.
        '''
        x_filtered = filter(lambda w:w in self.dicts['words2idx'].keys(), x.split())
        x_encoded = np.asarray([map(self.dicts['words2idx'].get,x_filtered)])
        pred_encoded = map(np.argmax,self.model.predict_on_batch(x_encoded)[0])
        return map(self.dicts['idx2labels'].get, pred_encoded)

In [45]:
data = load_atis(path='/Users/jacobsw/Desktop/IMPLEMENTATION_CAMP/CODE/OJO/LUIS/DATA/')
model_dir = '/Users/jacobsw/Desktop/IMPLEMENTATION_CAMP/CODE/OJO/ojo_ner/ojo_ner/models/'

In [46]:
ner = NER(data,model_dir=model_dir)

In [47]:
ner.train(validation=1,validation_freq=1000,verbose=1,verbose_freq=500,nepochs=20)

... configuring model
... building model
... training model
... Epoch: 1
Validation: Loss = 3.887879 | Accuracy = 0.646422
Test: Loss = 3.930928 | Accuracy = 0.619585
    ... trained 500 sentences
    ... trained 1000 sentences
Validation: Loss = 1.241905 | Accuracy = 0.728966
Test: Loss = 1.486263 | Accuracy = 0.674105
    ... trained 1500 sentences
    ... trained 2000 sentences
Validation: Loss = 0.976014 | Accuracy = 0.787186
Test: Loss = 1.241724 | Accuracy = 0.747876
    ... trained 2500 sentences
    ... trained 3000 sentences
Validation: Loss = 0.731050 | Accuracy = 0.836154
Test: Loss = 1.029356 | Accuracy = 0.775937
    ... trained 3500 sentences
... Epoch: 2
    ... trained 500 sentences
    ... trained 1000 sentences
    ... trained 1500 sentences
    ... trained 2000 sentences
Validation: Loss = 0.625033 | Accuracy = 0.862521
Test: Loss = 0.761316 | Accuracy = 0.829458
    ... trained 2500 sentences
    ... trained 3000 sentences
Validation: Loss = 0.593400 | Accuracy = 0.

In [48]:
sent = "i'd like to book a flight from charlotte to las vegas"
print ner.predict(sent)

['O', 'O', 'O', 'O', 'O', 'O', 'B-fromloc.city_name', 'O', 'B-toloc.city_name', 'I-toloc.city_name']
